In [1]:
%cd ..

/Users/mac/Monthy/Crawl_buffett


In [21]:
import pdfplumber
import pandas as pd
import numpy as np
import os, tabula, camelot, re
from datetime import datetime
from src.ocrpdf import ocrPDF

class GetVolume():
    def __init__(self, 
                 path_save="tests/"):
        self.path_save = path_save

    def openPdf(self, 
                path: str = "tests/Data/1301/PDF/2022_Q1_決算短信(2022_8_5).pdf"): 
        pdf = pdfplumber.open(path)
        return pdf

    def findText(self,
                    pdf,
                    text: str = "期末発行済株式数"):
        for page in range(len(pdf.pages)):
            text = pdf.pages[page].extract_text()
            text = text.replace('\n', ' ').replace(' ', '').replace('.', ',')
            text_first = '期末発行済株式数'
            id_first = text.find(text_first)
            if id_first >= 0:
                text = text[id_first:]
                return text

    def findData(self,
                 path: str = "tests/Data/1301/PDF/2022_Q1_決算短信(2022_8_5).pdf"):
        pdf = self.openPdf(path)
        text = self.findText(pdf)
        numbers = re.findall(r"\d{1,3}(?:,\d{3})*", text)
        numbers = [int(number.replace(",", "")) for number in numbers if int(number.replace(",", "")) > 3000]
        # print(numbers[0], numbers[2])
        return [numbers[0], numbers[2]]
    
    def getDataFromPdf(self,
                        id_company: int = 1301,
                        year: int = 2022,
                        quy: str = "Q1",):
        for file in os.listdir(self.path_save + f"Data/{id_company}/PDF"):
            if file.startswith(f"{year}_{quy}") and ("(訂正)" not in file) and '_ocr' not in file:
                file_name = file
                input_path = self.path_save + f"Data/{id_company}/PDF/{file_name}"
                date_volume = file_name[file_name.find("(") + 1 : file_name.find(")")]
                try:
                    lst_data_of_time = self.findData(input_path)
                except:
                    # try:
                        path_pdf_ocr = input_path.replace(".pdf", "_ocr.pdf")
                        print('Need ocr', path_pdf_ocr)
                        if not os.path.exists(path_pdf_ocr):
                            ocrPDF(input_path)
                        lst_data_of_time = self.findData(path_pdf_ocr)
                    # except:
                    #     lst_data_of_time = ["N/A", "N/A"]
                print(f"{year}_{quy}: {lst_data_of_time}")
                return [date_volume] + lst_data_of_time
        return ["N/A", "N/A", "N/A"]
    

    def getVolume(self,
                  id_company: int = 1301,
                  return_df: bool = False,
                  save_file: bool = True,):

        df_volume = pd.DataFrame(columns=["time", "date", "vol1", "vol2"])
        df = pd.read_csv(self.path_save + f"Data/{id_company}/docs/link.csv")
        for quy in ["Q1", "Q2", "Q3", "Q4"]:
            for id in df.index:
                year = df[f"Year"][id]
                df_volume.loc[(len(df_volume))] = [f"{year}_{quy}"] + self.getDataFromPdf(
                    id_company, year, quy)
        if save_file:
            df_volume.to_csv(self.path_save + f"Data/{id_company}/docs/volume.csv", index=False)
        if return_df:
            return df_volume

get_volume = GetVolume()
get_volume.getVolume(
    id_company=9997,
    return_df=True, 
    save_file=False)

2022_Q1: [94244472, 559257]
2021_Q1: [97244472, 571892]
2020_Q1: [97244472, 587411]
2019_Q1: [97244472, 547210]
2018_Q1: [97236456, 97235670]
2017_Q1: [97236456, 97236328]
2016_Q1: [113184548, 15947744]
2015_Q1: [113184548, 15947119]
2014_Q1: [113184548, 15945962]
Need ocr tests/Data/9997/PDF/2013_Q1_決算短信(2013_8_2)_ocr.pdf
2013_Q1: [56592274, 7972451]
2012_Q1: [56592274, 7657719]
2011_Q1: [56592274, 6467479]
2010_Q1: [56592274, 6466786]
2009_Q1: [56592274, 6466290]
2008_Q1: [56592274, 5465128]
2022_Q2: [97244472, 554132]
2021_Q2: [97244472, 559121]
2020_Q2: [97244472, 571586]
2019_Q2: [97244472, 587185]
2018_Q2: [97244472, 97237271]
2017_Q2: [97236456, 97236197]
2016_Q2: [113184548, 15947812]
2015_Q2: [113184548, 15947496]
2014_Q2: [113184548, 15946404]
Need ocr tests/Data/9997/PDF/2013_Q2_決算短信(2013_11_1)_ocr.pdf
2013_Q2: [113184548, 15945222]
Need ocr tests/Data/9997/PDF/2012_Q2_決算短信(2012_11_2)_ocr.pdf
2012_Q2: [56592274, 7911619]
2011_Q2: [56592274, 6995339]
Need ocr tests/Data/9997/


The program 'tesseract' could not be executed or was not found on your
system PATH.



Need ocr tests/Data/9997/PDF/2012_Q3_決算短信(2013_2_4)_ocr.pdf


MissingDependencyError: Could not find program 'tesseract' on the PATH

# test

In [23]:
pdf = get_volume.openPdf(path='tests/Data/9997/PDF/2012_Q3_決算短信(2013_2_4).pdf')

In [24]:
text = pdf.pages[1].extract_text()
text_first = '期末発行済株式数'
id_first = text.find(text_first)

print(text.replace('\n', ' ').replace(' ', '').replace('.', ','))

(cid:2)(cid:2)(cid:810)(cid:2)(cid:7496)(cid:11813)(cid:2640)(cid:13764)(cid:14804)(cid:14813)(cid:14805)(cid:5296)(cid:4030)(cid:3496)(cid:6588)(cid:12650)(cid:9919)(cid:9893)(cid:11800)(cid:6588)(cid:13382)(cid:2022)(cid:1989)(cid:1996)(cid:2054)(cid:12928)(cid:11728)(cid:2021)(cid:4662)(cid:2739)(cid:9248)(cid:2025)(cid:8675)(cid:3399)(cid:14822)(cid:8053)(cid:14967)(cid:2)(cid:2)(cid:10)(cid:12650)(cid:9919)(cid:9656)(cid:4048)(cid:2025)(cid:4364)(cid:6557)(cid:2061)(cid:2747)(cid:1985)(cid:8228)(cid:4719)(cid:4662)(cid:2739)(cid:9248)(cid:2025)(cid:8675)(cid:3399)(cid:11)(cid:14967)(cid:6338)(cid:11737)(cid:14809)(cid:9248)(cid:14804)(cid:9248)(cid:3628)(cid:14805)(cid:1940)(cid:13488)(cid:4374)(cid:14809)(cid:9248)(cid:14804)(cid:9248)(cid:3628)(cid:14805)(cid:14967)(cid:14967)(cid:2)(cid:2)(cid:14804)(cid:14814)(cid:14805)(cid:4030)(cid:3496)(cid:6588)(cid:12650)(cid:9919)(cid:12151)(cid:3403)(cid:11977)(cid:11563)(cid:2025)(cid:2782)(cid:5777)(cid:2022)(cid:8228)(cid:6573)(cid:

In [14]:
get_volume.findText(pdf)